## Deploy Annual Reports Mgmt Discussion Analysis Model Package from AWS Marketplace 


The solution uses text matching, NLP and Machine Learning to localize and identify management discussions. This will help financial and equity analysts to automatically run analytical models on unstructured text and draw insights from them.

This sample notebook shows you how to deploy Annual Reports Mgmt Discussion Analysis using Amazon SageMaker.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

#### Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. To deploy this ML model successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to Explainable AI: Algorithm Bias Detection. If so, skip step: [Subscribe to the model package](#1.-Subscribe-to-the-model-package)

#### Contents:
1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Create input payload](#B.-Create-input-payload)
   3. [Perform real-time inference](#C.-Perform-real-time-inference)
   4. [Visualize output](#D.-Visualize-output)
   5. [Delete the endpoint](#E.-Delete-the-endpoint)
3. [Perform batch inference](#3.-Perform-batch-inference) 
4. [Clean-up](#4.-Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    2. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))
    

#### Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

### 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page **Explainable AI: Algorithm Bias Detection**
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

In [1]:
model_package_arn='arn:aws:sagemaker:us-east-2:786796469737:model-package/mphasis-marketplace-financial-content-extraction'

In [2]:
import base64
import json 
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
from sagemaker import ModelPackage
#from urllib.parse import urlparse
import boto3
from IPython.display import Image
from PIL import Image as ImageEdit
#import urllib.request
import numpy as np

No handlers could be found for logger "sagemaker"


In [3]:
role = get_execution_role()

sagemaker_session = sage.Session()

bucket=sagemaker_session.default_bucket()
bucket

'sagemaker-us-east-2-786796469737'

### 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [4]:
model_name='mgmt-discussions-analyzer'

content_type='application/pdf'

real_time_inference_instance_type='ml.m5.xlarge'
batch_transform_inference_instance_type='ml.m5.large'

#### A. Create an endpoint

In [5]:

def predict_wrapper(endpoint, session):
    return sage.RealTimePredictor(endpoint, session,content_type)

#create a deployable model from the model package.
model = ModelPackage(role=role,
                    model_package_arn=model_package_arn,
                    sagemaker_session=sagemaker_session,
                    predictor_cls=predict_wrapper)

#Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

-----------!

Once endpoint has been created, you would be able to perform real-time inference.

#### B. Create input payload

In [6]:

# To show pdf-s inside ipython/jupyter notebooks you can use IFrame
from IPython.display import IFrame
IFrame("./Input/Sample.pdf", width=600, height=300)

<Add code snippet that shows the payload contents>

In [7]:
file_name = "./Input/Sample.pdf"
output_file_name = 'output.json'

#### C. Perform real-time inference

In [8]:
!aws sagemaker-runtime invoke-endpoint \
    --endpoint-name 'mgmt-discussions-analyzer' \
    --body fileb://$file_name \
    --content-type 'application/pdf' \
    --region us-east-2 \
    output.json

{
    "InvokedProductionVariant": "AllTraffic", 
    "ContentType": "application/json"
}


#### D. Visualize output

In [11]:
import json
with open('./output.json') as f:
    data = json.load(f)

print("Output: ")

print(data)

Output: 
{u'Credit file': {}, u'slashed prices': {}, u'Out of business': {}, u'Bankrupt': {}, u'Profit': {u'para-num-4': {u'count': 2, u'page_number': u'page72', u'para': u'Sitting fee indicated above also includes payment for Board-level committee meetings.The above remuneration of Executive Directors does not include provision for incremental liability on account of pension, gratuity, compensated absences and long service awards since actuarial valuation is done for the Company as a whole. The appointment of executive directors is by virtue of their employment with the Company as management employees and therefore, their terms of employment are governed by the applicable policies at the relevant point in time. Commission is subject to adequate profits being earned. A fair portion of the Executive Director\u2019s Performance Linked Incentive is linked to Company\u2019s performance. This creates alignment with the strategy and business priorities to enhance shareholder value. The total

#### E. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [14]:
predictor=sage.RealTimePredictor(model_name, sagemaker_session,content_type)
predictor.delete_endpoint(delete_endpoint_config=True)

### 3. Perform batch inference

In this section, you will perform batch inference using multiple input payloads together. If you are not familiar with batch transform, and want to learn more, see these links:
1. [How it works](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-batch-transform.html)
2. [How to run a batch transform job](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-batch.html)

In [15]:
#upload the batch-transform job input files to S3
transform_input_folder = "Input"
transform_input = sagemaker_session.upload_data(transform_input_folder, key_prefix=model_name) 
print("Transform input uploaded to " + transform_input)

Transform input uploaded to s3://sagemaker-us-east-2-786796469737/mgmt-discussions-analyzer


In [17]:
#Run the batch-transform job
transformer = model.transformer(1, 'ml.m5.4xlarge')
transformer.transform(transform_input, content_type=content_type)
transformer.wait()

....................../opt/program/tika/tika-server-1.9.jar
 * Serving Flask app "serve" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
 * Running on http://0.0.0.0:8080/ (Press CTRL+C to quit)
 * Restarting with stat
/opt/program/tika/tika-server-1.9.jar
 * Debugger is active!
 * Debugger PIN: 205-004-546
169.254.255.130 - - [01/Feb/2021 07:00:25] "#033GET /ping HTTP/1.1#033" 200 -
169.254.255.130 - - [01/Feb/2021 07:00:25] "#033GET /execution-parameters HTTP/1.1#033" 404 -
input/sample.pdf
Heading extraction complete
<B>NESTL&#XC9; INDIA LIMITED</B>
['<b>Directors&#x2019; report - 2014</b>']
starting page---- 24
Ending Page---- 34
['<b>ANNEXURE - 1 TO THE DIRECTORS&#x2019; REPORT</b>']
starting page---- 69
Ending Page---- 77
['<b>ANNEXURE &#x2013; 2 TO THE DIRECTORS&#x2019; REPORT</b>']
starting page---- 78
Ending Page---- 81
['<b>ANNEXURE - 3 TO THE DIRECTORS&#x2019; REPORT</b>']
starting page---- 82
Ending Page---- 84
{'heading-

In [18]:
#output is available on following path
transformer.output_path

's3://sagemaker-us-east-2-786796469737/mphasis-marketplace-financial-content-e-2021-02-01-06-56-51-895'

In [19]:
s3_conn = boto3.client("s3")
bucket_name="sagemaker-us-east-2-786796469737"
with open('./output.json', 'wb') as f:
    s3_conn.download_fileobj(bucket_name, 'mphasis-marketplace-financial-content-e-2021-02-01-06-56-51-895'+'/Sample.pdf.out', f)
    print("Output file loaded from bucket")

Output file loaded from bucket


In [21]:
output = open("./output.json","r+")
#print(output.read())

# copy and paste the output below in a browser tab to see the result
data = output.read()
print(data)

{"Acquire": {}, "Business Failure": {}, "credit": {}, "Dun & Bradstreet": {}, "Acquisition": {}, "cancellation": {}, "Credit file": {}, "earnings guidance": {}, "Acquisitive interest": {}, "Cash Collateral": {}, "Credit rating": {}, "earnings release": {}, "Adequate protection": {}, "Charge": {"para-num-0": {"page_number": "page72", "para": "The Committee oversees corporate social responsibility, corporate governance and other business related matters which may be referred by the Board or the Chairman, as and when deemed necessary, for the consideration and recommendation of the Committee. This Committee also discharges the role of Corporate Social Responsibility Committee under Section 135 of the Companies Act, 2013 which includes formulating and recommending to the Board, a Corporate Social Responsibility (CSR) Policy indicating the activities to be undertaken by the Company as per Schedule VII to the Companies Act, 2013; recommending the amount of expenditure to be incurred; and mon

### 4. Clean-up

#### A. Delete the model

In [22]:
model.delete_model()

#### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

